<a href="https://colab.research.google.com/github/laresamdeola/Machine-Learning/blob/main/Network_Security_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import math

In [157]:
data = pd.read_csv('UNSW-NB15_1.csv')

<ipython-input-157-adde79575085>:1: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('UNSW-NB15_1.csv')


Preprocessing

In [40]:
data.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,NaN,0


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 49 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   srcip             700000 non-null  object 
 1   sport             700000 non-null  object 
 2   dstip             700000 non-null  object 
 3   dsport            700000 non-null  object 
 4   proto             700000 non-null  object 
 5   state             700000 non-null  object 
 6   dur               700000 non-null  float64
 7   sbytes            700000 non-null  int64  
 8   dbytes            700000 non-null  int64  
 9   sttl              700000 non-null  int64  
 10  dttl              700000 non-null  int64  
 11  sloss             700000 non-null  int64  
 12  dloss             700000 non-null  int64  
 13  service           700000 non-null  object 
 14  Sload             700000 non-null  float64
 15  Dload             700000 non-null  float64
 16  Spkts             70

In [120]:
data.loc[:, 'srcip']

0         59.166.0.0
1         59.166.0.6
2         59.166.0.5
3         59.166.0.3
4         59.166.0.0
             ...    
699995    59.166.0.8
699996    59.166.0.0
699997    59.166.0.0
699998    59.166.0.6
699999    59.166.0.0
Name: srcip, Length: 700000, dtype: object

In [44]:
data['attack_cat'].fillna('normal', inplace=True)

In [58]:
data.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object')

In [69]:
data['service'].isnull().sum()

0

In [73]:
data['service']

0            -
1          dns
2          dns
3          dns
4            -
          ... 
699995       -
699996    http
699997       -
699998       -
699999       -
Name: service, Length: 700000, dtype: object

In [74]:
data['service'].replace('-', 'others', inplace=True)

In [76]:
data['service']

0         others
1            dns
2            dns
3            dns
4         others
           ...  
699995    others
699996      http
699997    others
699998    others
699999    others
Name: service, Length: 700000, dtype: object

In [61]:
data['dsport']

0          1024
1            53
2            53
3            53
4           111
          ...  
699995    31010
699996       80
699997     5190
699998      111
699999    54145
Name: dsport, Length: 700000, dtype: object

In [118]:
data.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object')

In [107]:
data['is_sm_ips_ports'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 700000 entries, 0 to 699999
Series name: is_sm_ips_ports
Non-Null Count   Dtype
--------------   -----
700000 non-null  int64
dtypes: int64(1)
memory usage: 5.3 MB


In [108]:
data['is_sm_ips_ports']

0         0
1         0
2         0
3         0
4         0
         ..
699995    0
699996    0
699997    0
699998    0
699999    0
Name: is_sm_ips_ports, Length: 700000, dtype: int64

In [123]:
data['ct_ftp_cmd']

0         0
1         0
2         0
3         0
4         0
         ..
699995    0
699996    0
699997    0
699998    0
699999    0
Name: ct_ftp_cmd, Length: 700000, dtype: int64

In [87]:
data['state'].replace('-', 'others', inplace=True)

In [124]:
data['state'].empty

False

In [173]:
data.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object')

In [184]:
data['ct_dst_src_ltm']

0         2
1         1
2         1
3         1
4         2
         ..
699995    4
699996    2
699997    2
699998    3
699999    2
Name: ct_dst_src_ltm, Length: 700000, dtype: int64

In [125]:
data

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,normal,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,normal,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,normal,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,normal,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699995,59.166.0.8,12520,149.171.126.6,31010,tcp,FIN,0.020383,320,1874,31,...,0,8,20,7,5,1,1,4,normal,0
699996,59.166.0.0,18895,149.171.126.9,80,tcp,FIN,1.402957,19410,1087890,31,...,0,1,1,2,7,2,2,2,normal,0
699997,59.166.0.0,30103,149.171.126.5,5190,tcp,FIN,0.007108,2158,2464,31,...,0,13,13,6,7,2,1,2,normal,0
699998,59.166.0.6,30388,149.171.126.5,111,udp,CON,0.004435,568,304,31,...,0,10,13,6,5,1,1,3,normal,0


In [167]:
X = data.iloc[:, 0:49].values
y = data.iloc[:, -1].values

In [177]:
X[:, 4:5]

array([['udp'],
       ['udp'],
       ['udp'],
       ...,
       ['tcp'],
       ['udp'],
       ['tcp']], dtype=object)

In [178]:
X[:, 5:6]

array([['CON'],
       ['CON'],
       ['CON'],
       ...,
       ['FIN'],
       ['CON'],
       ['FIN']], dtype=object)

In [179]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()

# Encode the "proto" column
X[:, 4:5] = np.array(encoder.fit_transform(X[:, 4:5]))
X[:, 5:6] = np.array(encoder.fit_transform(X[:, 5:6]))

In [180]:
X[:, 4:5]

array([[120.0],
       [120.0],
       [120.0],
       ...,
       [114.0],
       [120.0],
       [114.0]], dtype=object)

In [181]:
X[:, 5:6]

array([[2.0],
       [2.0],
       [2.0],
       ...,
       [5.0],
       [2.0],
       [5.0]], dtype=object)

In [183]:
X[:, 47]

array([nan, nan, nan, ..., nan, nan, nan], dtype=object)

In [144]:
data

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,NaN,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,normal,0
1,NaN,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,normal,0
2,NaN,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,normal,0
3,NaN,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,normal,0
4,NaN,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699995,NaN,12520,149.171.126.6,31010,tcp,FIN,0.020383,320,1874,31,...,0,8,20,7,5,1,1,4,normal,0
699996,NaN,18895,149.171.126.9,80,tcp,FIN,1.402957,19410,1087890,31,...,0,1,1,2,7,2,2,2,normal,0
699997,NaN,30103,149.171.126.5,5190,tcp,FIN,0.007108,2158,2464,31,...,0,13,13,6,7,2,1,2,normal,0
699998,NaN,30388,149.171.126.5,111,udp,CON,0.004435,568,304,31,...,0,10,13,6,5,1,1,3,normal,0


In [140]:
len(data.columns)

49